In [ ]:
# %%%
import json
import pandas as pd

In [ ]:
# Define a function to load JSONL files into a DataFrame
def load_jsonl_to_dataframe(file_path, fields):
    data = []
    with open(file_path, "r") as f:
        for line in f:
            entry = json.loads(line)
            filtered_entry = {field: entry.get(field) for field in fields}
            data.append(filtered_entry)
    return pd.DataFrame(data)


# List of fields to include from each file
fields_questions = ["question_id", "text", "image"]
fields_responses = ["question_id", "text", "model_id"]
fields_review = ["id", "question_id", "content", "scores"]

# Load the files using the snakemake object
questions_df = load_jsonl_to_dataframe(snakemake.input.questions, fields_questions)

reference_responses_df = load_jsonl_to_dataframe(
    snakemake.input.reference_responses, fields_responses
)
cw_responses_df = load_jsonl_to_dataframe(
    snakemake.input.llava_responses[0], fields_responses
)
llava_responses_df = load_jsonl_to_dataframe(
    snakemake.input.llava_responses[1], fields_responses
)

gpt4transcriptome_responses_df = load_jsonl_to_dataframe(
    snakemake.input.gpt4transcriptome_baseline_responses, fields_responses
)
llava_eval_gpt4_review_df = load_jsonl_to_dataframe(
    snakemake.input.llava_eval_gpt4_review, fields_review
)

# Merge the responses on 'question_id'
responses = {"question": questions_df.set_index("question_id")["text"]}
for response in [
    reference_responses_df,
    cw_responses_df,
    llava_responses_df,
    gpt4transcriptome_responses_df,
]:
    assert response.model_id.nunique() == 1
    model_id = response.model_id.iloc[0]
    responses[f"response_{model_id}"] = response.set_index("question_id")["text"]


responses = pd.DataFrame(responses)
responses["review"] = llava_eval_gpt4_review_df.set_index("question_id")["content"]
responses

In [ ]:

score_df = pd.DataFrame.from_records(
    llava_eval_gpt4_review_df["scores"], index=llava_eval_gpt4_review_df.question_id
)
score_df.columns = [c.replace("generation_", "score_") for c in score_df.columns]

In [ ]:
responses.join(score_df).to_excel(snakemake.output.llava_eval_table)